In [25]:
import pandas as pd
from sqlalchemy import create_engine

In [26]:
baltimore_file = r"C:\Users\johns\Desktop\Crime-Data-ETL-Project\Resources\BPD_Part_1_Victim_Based_Crime_Data.csv"
baltimore_df = pd.read_csv(baltimore_file)
baltimore_df.head()

,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,District,Neighborhood,Longitude,Latitude,Location 1,Premise,vri_name1,Total Incidents
0,11/2/2019,21:00:00,6D,300 S STRICKER ST,LARCENY FROM AUTO,O,NaN,935,SOUTHERN,NEW SOUTHWEST/MOUNT CLARE,-76.640774,39.284127,NaN,STREET,Tri-District,1
1,11/2/2019,23:50:00,5A,1600 S CHARLES ST,BURGLARY,I,NaN,942,SOUTHERN,S.B.I.C.,-76.614083,39.271502,NaN,ROW/TOWNHOUSE-OCC,NaN,1
2,11/2/2019,19:00:00,6E,3700 2ND ST,LARCENY,O,NaN,913,SOUTHERN,BROOKLYN,-76.609071,39.237094,NaN,STREET,NaN,1
3,11/2/2019,18:00:00,7A,3500 NEWLAND RD,AUTO THEFT,O,NaN,525,NORTHERN,GUILFORD,-76.610575,39.332097,NaN,STREET,NaN,1
4,11/2/2019,17:04:37,9S,1000 HOMEWOOD AV,SHOOTING,Outside,FIREARM,313,EASTERN,JOHNSTON SQUARE,-76.606416,39.301422,NaN,STREET,NaN,1


In [27]:
# Create a filtered dataframe from specific columns
baltimore_columns = ["CrimeDate", "Location", "Description", "Premise"]
baltimore_transformed = baltimore_df[baltimore_columns].copy()
baltimore_transformed.head()

,CrimeDate,Location,Description,Premise
0,11/2/2019,300 S STRICKER ST,LARCENY FROM AUTO,STREET
1,11/2/2019,1600 S CHARLES ST,BURGLARY,ROW/TOWNHOUSE-OCC
2,11/2/2019,3700 2ND ST,LARCENY,STREET
3,11/2/2019,3500 NEWLAND RD,AUTO THEFT,STREET
4,11/2/2019,1000 HOMEWOOD AV,SHOOTING,STREET


In [28]:
#insert columns
baltimore_transformed.insert(2, "City", "BALTIMORE")
baltimore_transformed.insert(3, "State", "MARYLAND")
baltimore_transformed.head()

,CrimeDate,Location,City,State,Description,Premise
0,11/2/2019,300 S STRICKER ST,BALTIMORE,MARYLAND,LARCENY FROM AUTO,STREET
1,11/2/2019,1600 S CHARLES ST,BALTIMORE,MARYLAND,BURGLARY,ROW/TOWNHOUSE-OCC
2,11/2/2019,3700 2ND ST,BALTIMORE,MARYLAND,LARCENY,STREET
3,11/2/2019,3500 NEWLAND RD,BALTIMORE,MARYLAND,AUTO THEFT,STREET
4,11/2/2019,1000 HOMEWOOD AV,BALTIMORE,MARYLAND,SHOOTING,STREET


In [29]:
baltimore_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281729 entries, 0 to 281728
Data columns (total 6 columns):
CrimeDate      281729 non-null object
Location       280508 non-null object
City           281729 non-null object
State          281729 non-null object
Description    281729 non-null object
Premise        249964 non-null object
dtypes: object(6)
memory usage: 12.9+ MB


In [30]:
#convert columndate to date/time data type
baltimore_transformed["CrimeDate"] = pd.to_datetime(baltimore_transformed["CrimeDate"])

In [31]:
baltimore_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281729 entries, 0 to 281728
Data columns (total 6 columns):
CrimeDate      281729 non-null datetime64[ns]
Location       280508 non-null object
City           281729 non-null object
State          281729 non-null object
Description    281729 non-null object
Premise        249964 non-null object
dtypes: datetime64[ns](1), object(5)
memory usage: 12.9+ MB


In [32]:
#cut data down to years 2014-2017
start_date = "2014-01-01"
end_date = "2017-12-31"

In [33]:
filter_date = (baltimore_transformed["CrimeDate"] > start_date) & (baltimore_transformed["CrimeDate"] <= end_date)

In [34]:
baltimore_transformed = baltimore_transformed.loc[filter_date]
baltimore_transformed.head()

,CrimeDate,Location,City,State,Description,Premise
87222,2017-12-31,5000 RAINTREE WAY,BALTIMORE,MARYLAND,LARCENY,STREET
87223,2017-12-31,4500 ERDMAN AVE,BALTIMORE,MARYLAND,ROBBERY - COMMERCIAL,CONVENIENCE STORE
87224,2017-12-31,5000 HARFORD RD,BALTIMORE,MARYLAND,AGG. ASSAULT,GAS STATION
87225,2017-12-31,5000 HARFORD RD,BALTIMORE,MARYLAND,COMMON ASSAULT,NaN
87226,2017-12-31,5300 WRIGHT AVE,BALTIMORE,MARYLAND,AGG. ASSAULT,ROW/TOWNHOUSE-OCC


In [35]:
# Rename the column headers to load into database
baltimore_transformed = baltimore_transformed.rename(columns={"CrimeDate": "crime_date","Location": "location", "City": "city",
                                                              "State": "state", "Description": "description", "Premise": "premise"})

In [36]:
baltimore_transformed.head()

,crime_date,location,city,state,description,premise
87222,2017-12-31,5000 RAINTREE WAY,BALTIMORE,MARYLAND,LARCENY,STREET
87223,2017-12-31,4500 ERDMAN AVE,BALTIMORE,MARYLAND,ROBBERY - COMMERCIAL,CONVENIENCE STORE
87224,2017-12-31,5000 HARFORD RD,BALTIMORE,MARYLAND,AGG. ASSAULT,GAS STATION
87225,2017-12-31,5000 HARFORD RD,BALTIMORE,MARYLAND,COMMON ASSAULT,NaN
87226,2017-12-31,5300 WRIGHT AVE,BALTIMORE,MARYLAND,AGG. ASSAULT,ROW/TOWNHOUSE-OCC


In [13]:
#create database connection
connection_string = "postgres:2231jack.@localhost:5432/Crime_Data"
engine = create_engine(f'postgresql://{connection_string}')

In [14]:
engine.table_names()

['baltimore', 'orlando']

In [20]:
baltimore_transformed.to_sql(name="baltimore", con=engine, if_exists="append", index= False)